# **Exploratory Data Analysis of Climatic Variables**

In [ ]:
# Libraries
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, spearmanr
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data
df = pd.read_csv('AVONETplusClim.csv' )

## **1. Body Size (Mass)**

First of all, distribution of mass is explored using histograms. For the raw data: 

In [94]:
# 1. Raw Data Plot (Histogram + Box)
fig1 = px.histogram(
    df, 
    x='Mass', 
    title='Raw Mass Distribution', 
    nbins=50, 
    marginal='box',
    color_discrete_sequence=['salmon']
)
fig1.update_layout(xaxis_title="Mass (g)", yaxis_title="Count")
fig1.show()

The data is extremely right-skewed. In addition, there are 11 outliers that disrupt the visualization (>10000). To approach a biologically meaningful distribution and improve the visualization, log-transformation is applied. Additionally, the outliers are excluded since they make up a negligible fraction of the dataset (11/9878):

In [95]:
# 2. Filter (< 10k) & Log Transform
df_clean = df[(df['Mass'] < 11500) & (df['Mass'] > 0)].copy()
df_clean['Log_Mass'] = np.log10(df_clean['Mass'])

# 3. Log Data Plot (Histogram + Box)
fig2 = px.histogram(
    df_clean, 
    x='Log_Mass', 
    title='Log10(Mass) Distribution (<10k)', 
    nbins=50, 
    marginal='box',
    color_discrete_sequence=['salmon']
)
fig2.update_layout(xaxis_title="Log10(Mass) (Log10(g))", yaxis_title="Count")
fig2.show()

The data is less but still visibly right-skewed. To assess the normality of distribution, skewness and kurtosis are calculated since Shapiro-Wilk test is unreliable for large sample sizes (>5000):

In [ ]:
skew_val = skew(df_clean['Log_Mass'])
kurt_val = kurtosis(df_clean['Log_Mass'])

print(f"Skewness: {skew_val:.4f}")
print(f"Kurtosis: {kurt_val:.4f}")

Both skewness and kurtosis fall between -1 and 1, thus the distribution is assumed to be normal.

### **1.1. Mass and Temperature**

Before starting with temperature, similar to mass, distribution of temperature is explored based on skewness and kurtosis. For minimum, maximum, and average temperatures:

In [114]:
fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=("Minimum Temperature (bio_6)", "Maximum Temperature (bio_5)", "Average Temperature (bio_1)")
)

# bio_6
fig.add_trace(
    go.Histogram(x=df_clean['bio_6'], nbinsx=50, name='bio_6', marker_color='blue'),
    row=1, col=1
)

# bio_5
fig.add_trace(
    go.Histogram(x=df_clean['bio_5'], nbinsx=50, name='bio_5', marker_color='red'),
    row=1, col=2
)

# bio_1
fig.add_trace(
    go.Histogram(x=df_clean['bio_1'], nbinsx=50, name='bio_1', marker_color='green'),
    row=1, col=3
)

fig.update_layout(
    title_text="Distributions of Temperature Variables",
    showlegend=False,
    bargap=0.1
)

fig.update_xaxes(title_text="Temperature (°C)", row=1, col=1)
fig.update_xaxes(title_text="Temperature (°C)", row=1, col=2)
fig.update_xaxes(title_text="Temperature (°C)", row=1, col=3)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

In [ ]:
print(f"{'Variable':<10} | {'Skewness':<10} | {'Kurtosis':<10}")

for var in ['bio_6', 'bio_5', 'bio_1']:
    s = skew(df_clean[var])
    k = kurtosis(df_clean[var])
    
    print(f"{var:<10} | {s:.4f}    | {k:.4f}")

Variable   | Skewness   | Kurtosis  
bio_6      | -1.592     | 2.474
bio_5      | -1.182     | 3.370
bio_1      | -1.679     | 3.026


The data is left-skewed. Attempts to normalize the data, such as shifting and squaring the values, could work but the biological meaning would be lost. Therefore, to be able to directly interpret the values, raw scale is kept for temperature variables. Consequently, rejecting a normal distribution, a non-parametric measure is used to assess correlation, which is Spearman's rank correlation coefficient.

For minimum temperature:

* H0: There is no correlation between mass and minimum temperature.
* H1: There is a correlation between mass and minimum temperature. 

In [ ]:
correlation_rho, p_value_spearman = spearmanr(df['bio_6'], df['Mass'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

Spearman rho: -0.0346
p-value: 5.9205e-04


For maximum temperature:

* H0: There is no correlation between mass and maximum temperature.
* H1: There is a correlation between mass and maximum temperature. 

In [ ]:
correlation_rho, p_value_spearman = spearmanr(df['bio_5'], df['Mass'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

Spearman rho: 0.0285
p-value: 4.6358e-03


For average temperature:

* H0: There is no correlation between mass and average temperature.
* H1: There is a correlation between mass and average temperature. 

In [ ]:
correlation_rho, p_value_spearman = spearmanr(df['bio_1'], df['Mass'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

Spearman rho: -0.0120
p-value: 2.3125e-01


For minimum and maximum temperature, p < 0.05, thus the null hypotheses are rejected. There are statistically significant relationships between mass-minimum temperature and mass-maximum temperature. However, for average temperature, p > 0.05, thus the null hypothesis is failed to reject. To illustrate the relationships, scatter plots are created:

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import spearmanr

# 1. Veriyi Yükle ve Temizle
df = pd.read_csv('AVONET_data.csv')
df_clean = df[(df['Mass'] < 10000) & (df['Mass'] > 0)].copy()
df_clean['Log_Mass'] = np.log10(df_clean['Mass'])

# Renk Ayarları (İsteğine Göre)
color_map = {
    'bio_6': 'blue',   # Minimum -> Mavi
    'bio_5': 'red',    # Maximum -> Kırmızı
    'bio_1': 'green'   # Average -> Yeşil
}

titles = ["Minimum Temperature (bio_6)", "Maximum Temperature (bio_5)", "Average Temperature (bio_1)"]
vars_list = ['bio_6', 'bio_5', 'bio_1']

# 2. Yan Yana 3 Grafik (Subplots)
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=titles,
    horizontal_spacing=0.05
)

for i, var in enumerate(vars_list):
    x_data = df_clean[var]
    y_data = df_clean['Log_Mass']
    
    # NaN temizliği (polyfit ve spearman hata vermesin diye)
    mask = ~np.isnan(x_data) & ~np.isnan(y_data)
    x_data = x_data[mask]
    y_data = y_data[mask]

    # İstatistikler
    rho, p_val = spearmanr(x_data, y_data)
    p_text = "< 0.001" if p_val < 0.001 else f"{p_val:.3f}"

    # A) Scatter Plot (Noktalar) - Özel Renkler
    fig.add_trace(
        go.Scatter(
            x=x_data, y=y_data, 
            mode='markers', 
            name='Data',
            marker=dict(size=3, color=color_map[var], opacity=0.3),
            showlegend=False
        ),
        row=1, col=i+1
    )

    # B) Trendline (Regresyon Çizgisi) - Mor Renk
    m, b = np.polyfit(x_data, y_data, 1)
    x_line = np.linspace(x_data.min(), x_data.max(), 100)
    y_line = m * x_line + b
    
    fig.add_trace(
        go.Scatter(
            x=x_line, y=y_line, 
            mode='lines', 
            name='Trend',
            line=dict(color='purple', width=2), # Trendline MOR oldu
            showlegend=False
        ),
        row=1, col=i+1
    )

    # C) İstatistik Kutusu
    axis_idx = "" if i == 0 else i + 1
    fig.add_annotation(
        text=f"<b>Spearman ρ: {rho:.3f}</b><br>p: {p_text}",
        xref=f"x{axis_idx} domain", 
        yref=f"y{axis_idx} domain",
        x=0.95, y=0.95,
        showarrow=False,
        align="right",
        bgcolor="rgba(255, 255, 255, 0.8)",
        bordercolor="black",
        borderwidth=1
    )

    fig.update_xaxes(title_text="Temperature (°C)", row=1, col=i+1)

# Genel Ayarlar
fig.update_yaxes(title_text="Log10(Mass)", row=1, col=1)
fig.update_layout(
    title_text="Log10(Mass) vs Temperature Variables",
    height=500,
    width=1200,
    template="plotly_white"
)

fig.show()

### **1.2. Mass and Precipitation**

Since Mass is not normally distributed, Spearman test is used, in which

* H0: There is no monotonic relationship between Mass and Precipitation.
* H1: There is a monotonic relationship between Mass and Precipitation.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_12'], df['Mass'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

Since p < 0.05, the null hypothesis (H0) is rejected. There is a statistically significant relationship between Mass and Precipitation. To illustrate the relationship, a scatter plot is created (with log(Mass) for better visualization).

In [ ]:
df['log_Mass'] = np.log10(df['Mass'])

rho, p = stats.spearmanr(df['bio_12'], df['log_Mass'], nan_policy='omit')

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

sns.regplot(
    x='bio_12', 
    y='log_Mass', 
    data=df, 
    ax=ax, 
    scatter_kws={'alpha': 0.1, 's': 10}, 
    line_kws={'color': 'red', 'lw': 2, 'label': f'ρ = {rho:.3f}, p = {p:.2e}'}
)

ax.set_title(f"Log(Mass) vs. Precipitation (mm)", fontsize=14)
ax.set_xlabel(f"Precipitation")
ax.set_ylabel(f"Log(Mass) (log(g))") 
ax.legend(loc='upper right', fontsize=10)

plt.show()

## **2. Leg Size (Tarsus Length)**

### **2.1. Tarsus Length and Temperature**

To choose the proper statistical test, Shapiro-Wilk test is done, in which

* H0: The Tarsus Length is normally distributed.
* H1: The Tarsus Length is not normally distributed.

In [ ]:
shapiro_w, shapiro_p = stats.shapiro(df['Tarsus.Length'])

print(f"N: {len(df['Tarsus.Length'])}")
print(f"p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as Pearson cannot be used. As an alternative, Spearman test is used, in which

* H0: There is no monotonic relationship between Tarsus Length and Minimum temperature.
* H1: There is a monotonic relationship between Tarsus Length and Minimum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_6'], df['Tarsus.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between Tarsus Length and Maximum temperature.
* H1: There is a monotonic relationship between Tarsus Length and Maximum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_5'], df['Tarsus.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between Tarsus Length and Average temperature.
* H1: There is a monotonic relationship between Tarsus Length and Average temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_1'], df['Tarsus.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

For minimum, maximum, and average temperature, p << 0.05, thus the null hypothesis (H0) is rejected. Overall, there is a statistically significant relationship between tarsus length and temperature. Finally, to illustrate the relationship, scatter plots are created (with log(Length) for better visualization).

In [ ]:
temp_cols = ['bio_6', 'bio_1', 'bio_5']

df['log_TarsusLength'] = np.log10(df['Tarsus.Length'])
df_temp = df[['log_TarsusLength'] + temp_cols].copy()
df_melted = df_temp.melt(id_vars='log_TarsusLength', var_name='Temperature_Type', value_name='Temperature_Value')

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Log(Tarsus Length) vs. Temperature', fontsize=18)

label_map = {
    'bio_6': 'Minimum Temperature', 
    'bio_1': 'Average Temperature', 
    'bio_5': 'Maximum Temperature'
}

for i, col in enumerate(temp_cols):
    subset = df_melted[df_melted['Temperature_Type'] == col]

    rho, p = stats.spearmanr(subset['Temperature_Value'], subset['log_TarsusLength'])

    sns.regplot(x='Temperature_Value', y='log_TarsusLength', data=subset, ax=axes[i], 
        scatter_kws={'alpha': 0.1, 's': 10}, 
            line_kws={'color': 'red', 'lw': 2, 'label': f'Spearman Trend Line (ρ = {rho:.3f}, p = {p:.2e})'})

    axes[i].set_title(f"{label_map[col]}", fontsize=14)
    axes[i].set_xlabel(f"{label_map[col]} (°C)")
    axes[i].set_ylabel("Log(Length) (log(mm))" if i == 0 else "") 
    axes[i].legend(loc='upper right', fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## **3. Tail Size (Tail Length)**

### **3.1. Tail Length and Temperature**

To choose the proper statistical test, Shapiro-Wilk test is done, in which

* H0: The Tail Length is normally distributed.
* H1: The Tail Length is not normally distributed.

In [ ]:
shapiro_w, shapiro_p = stats.shapiro(df['Tail.Length'])

print(f"N: {len(df['Tail.Length'])}")
print(f"p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as Pearson cannot be used. As an alternative, Spearman test is used, in which

* H0: There is no monotonic relationship between Tail Length and Minimum temperature.
* H1: There is a monotonic relationship between Tail Length and Minimum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_6'], df['Tail.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between Tail Length and Maximum temperature.
* H1: There is a monotonic relationship between Tail Length and Maximum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_5'], df['Tail.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between Tail Length and Average temperature.
* H1: There is a monotonic relationship between Tail Length and Average temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_1'], df['Tail.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

For minimum, maximum, and average temperature, p < 0.05, thus the null hypothesis (H0) is rejected. Overall, there is a statistically significant relationship between tail length and temperature. Finally, to illustrate the relationship, scatter plots are created (with log(Length) for better visualization).

In [ ]:
temp_cols = ['bio_6', 'bio_1', 'bio_5']

df['log_TailLength'] = np.log10(df['Tail.Length'])
df_temp = df[['log_TailLength'] + temp_cols].copy()
df_melted = df_temp.melt(id_vars='log_TailLength', var_name='Temperature_Type', value_name='Temperature_Value')

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Log(Tail Length) vs. Temperature', fontsize=18)

label_map = {
    'bio_6': 'Minimum Temperature', 
    'bio_1': 'Average Temperature', 
    'bio_5': 'Maximum Temperature'
}

for i, col in enumerate(temp_cols):
    subset = df_melted[df_melted['Temperature_Type'] == col]

    rho, p = stats.spearmanr(subset['Temperature_Value'], subset['log_TailLength'])

    sns.regplot(x='Temperature_Value', y='log_TailLength', data=subset, ax=axes[i], 
        scatter_kws={'alpha': 0.1, 's': 10}, 
            line_kws={'color': 'red', 'lw': 2, 'label': f'Spearman Trend Line (ρ = {rho:.3f}, p = {p:.2e})'})

    axes[i].set_title(f"{label_map[col]}", fontsize=14)
    axes[i].set_xlabel(f"{label_map[col]} (°C)")
    axes[i].set_ylabel("Log(Length) (log(mm))" if i == 0 else "") 
    axes[i].legend(loc='upper right', fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## **4. Wing Size (Wing Length and Hand-Wing Index)**

### **4.1. Wing Length and Temperature**

To choose the proper statistical test, Shapiro-Wilk test is done, in which

* H0: The Wing Length is normally distributed.
* H1: The Wing Length is not normally distributed.

In [ ]:
shapiro_w, shapiro_p = stats.shapiro(df['Wing.Length'])

print(f"N: {len(df['Wing.Length'])}")
print(f"p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as Pearson cannot be used. As an alternative, Spearman test is used, in which

* H0: There is no monotonic relationship between Wing Length and Minimum temperature.
* H1: There is a monotonic relationship between Wing Length and Minimum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_6'], df['Wing.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between Wing Length and Maximum temperature.
* H1: There is a monotonic relationship between Wing Length and Maximum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_5'], df['Wing.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between Wing Length and Average temperature.
* H1: There is a monotonic relationship between Wing Length and Average temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_1'], df['Wing.Length'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

For minimum, maximum, and average temperature, p < 0.05, thus the null hypothesis (H0) is rejected. Overall, there is a statistically significant relationship between tail length and temperature. Finally, to illustrate the relationship, scatter plots are created (with log(Length) for better visualization).

In [ ]:
temp_cols = ['bio_6', 'bio_1', 'bio_5']

df['log_WingLength'] = np.log10(df['Wing.Length'])
df_temp = df[['log_WingLength'] + temp_cols].copy()
df_melted = df_temp.melt(id_vars='log_WingLength', var_name='Temperature_Type', value_name='Temperature_Value')

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Log(Wing Length) vs. Temperature', fontsize=18)

label_map = {
    'bio_6': 'Minimum Temperature', 
    'bio_1': 'Average Temperature', 
    'bio_5': 'Maximum Temperature'
}

for i, col in enumerate(temp_cols):
    subset = df_melted[df_melted['Temperature_Type'] == col]

    rho, p = stats.spearmanr(subset['Temperature_Value'], subset['log_WingLength'])

    sns.regplot(x='Temperature_Value', y='log_WingLength', data=subset, ax=axes[i], 
        scatter_kws={'alpha': 0.1, 's': 10}, 
            line_kws={'color': 'red', 'lw': 2, 'label': f'Spearman Trend Line (ρ = {rho:.3f}, p = {p:.2e})'})

    axes[i].set_title(f"{label_map[col]}", fontsize=14)
    axes[i].set_xlabel(f"{label_map[col]} (°C)")
    axes[i].set_ylabel("Log(Length) (log(mm))" if i == 0 else "") 
    axes[i].legend(loc='upper right', fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### **4.2. Hand-Wing Index (HWI) and Temperature**

To choose the proper statistical test, Shapiro-Wilk test is done, in which

* H0: The HWI is normally distributed.
* H1: The HWI is not normally distributed.

In [ ]:
shapiro_w, shapiro_p = stats.shapiro(df['Hand-Wing.Index'])

print(f"N: {len(df['Hand-Wing.Index'])}")
print(f"p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as Pearson cannot be used. As an alternative, Spearman test is used, in which

* H0: There is no monotonic relationship between HWI and Minimum temperature.
* H1: There is a monotonic relationship between HWI and Minimum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_6'], df['Hand-Wing.Index'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between HWI and Maximum temperature.
* H1: There is a monotonic relationship between HWI and Maximum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_5'], df['Hand-Wing.Index'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between HWI and Average temperature.
* H1: There is a monotonic relationship between HWI and Average temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_1'], df['Hand-Wing.Index'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

For minimum and average temperature, p < 0.05, thus the null hypothesis (H0) is rejected. There is a statistically significant relationship between mass-minimum temperature and mass-average temperature. However, for maximum temperature, p > 0.05, thus the null hypothesis is failed to reject. Finally, to illustrate the relationship, scatter plots are created (with log() for better visualization).

In [ ]:
temp_cols = ['bio_6', 'bio_1', 'bio_5']

df['log_HWI'] = np.log10(df['Hand-Wing.Index'])
df_temp = df[['log_HWI'] + temp_cols].copy()
df_melted = df_temp.melt(id_vars='log_HWI', var_name='Temperature_Type', value_name='Temperature_Value')

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Log(HWI) vs. Temperature', fontsize=18)

label_map = {
    'bio_6': 'Minimum Temperature', 
    'bio_1': 'Average Temperature', 
    'bio_5': 'Maximum Temperature'
}

for i, col in enumerate(temp_cols):
    subset = df_melted[df_melted['Temperature_Type'] == col]

    rho, p = stats.spearmanr(subset['Temperature_Value'], subset['log_HWI'])

    sns.regplot(x='Temperature_Value', y='log_HWI', data=subset, ax=axes[i], 
        scatter_kws={'alpha': 0.1, 's': 10}, 
            line_kws={'color': 'red', 'lw': 2, 'label': f'Spearman Trend Line (ρ = {rho:.3f}, p = {p:.2e})'})

    axes[i].set_title(f"{label_map[col]}", fontsize=14)
    axes[i].set_xlabel(f"{label_map[col]} (°C)")
    axes[i].set_ylabel("Log(HWI) (log())" if i == 0 else "") 
    axes[i].legend(loc='upper right', fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## **5. Beak Size**

### **5.1. Beak Length and Temperature**

To choose the proper statistical test, Shapiro-Wilk test is done, in which

* H0: The Beak Length is normally distributed.
* H1: The Beak Length is not normally distributed.

In [ ]:
shapiro_w, shapiro_p = stats.shapiro(df['Beak.Length_Culmen'])

print(f"N: {len(df['Beak.Length_Culmen'])}")
print(f"p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as Pearson cannot be used. As an alternative, Spearman test is used, in which

* H0: There is no monotonic relationship between Beak Length and Minimum temperature.
* H1: There is a monotonic relationship between Beak Length and Minimum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_6'], df['Beak.Length_Culmen'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between Beak Length and Maximum temperature.
* H1: There is a monotonic relationship between Beak Length and Maximum temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_5'], df['Beak.Length_Culmen'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

* H0: There is no monotonic relationship between Beak Length and Average temperature.
* H1: There is a monotonic relationship between Beak Length and Average temperature.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_1'], df['Beak.Length_Culmen'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

For minimum, maximum, and average temperature, p < 0.05, thus the null hypothesis (H0) is rejected. Overall, there is a statistically significant relationship between beak length and temperature. Finally, to illustrate the relationship, scatter plots are created (with log(Length) for better visualization).

In [ ]:
temp_cols = ['bio_6', 'bio_1', 'bio_5']

df['log_BeakLength'] = np.log10(df['Beak.Length_Culmen'])
df_temp = df[['log_BeakLength'] + temp_cols].copy()
df_melted = df_temp.melt(id_vars='log_BeakLength', var_name='Temperature_Type', value_name='Temperature_Value')

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Log(Beak Length) vs. Temperature', fontsize=18)

label_map = {
    'bio_6': 'Minimum Temperature', 
    'bio_1': 'Average Temperature', 
    'bio_5': 'Maximum Temperature'
}

for i, col in enumerate(temp_cols):
    subset = df_melted[df_melted['Temperature_Type'] == col]

    rho, p = stats.spearmanr(subset['Temperature_Value'], subset['log_BeakLength'])

    sns.regplot(x='Temperature_Value', y='log_BeakLength', data=subset, ax=axes[i], 
        scatter_kws={'alpha': 0.1, 's': 10}, 
            line_kws={'color': 'red', 'lw': 2, 'label': f'Spearman Trend Line (ρ = {rho:.3f}, p = {p:.2e})'})

    axes[i].set_title(f"{label_map[col]}", fontsize=14)
    axes[i].set_xlabel(f"{label_map[col]} (°C)")
    axes[i].set_ylabel("Log(Length) (log(mm))" if i == 0 else "") 
    axes[i].legend(loc='upper right', fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### **5.2. Beak Size and Precipitation**

Since Beak Length is not normally distributed, Spearman test is used, in which

* H0: There is no monotonic relationship between Beak Length and Precipitation.
* H1: There is a monotonic relationship between Beak Length and Precipitation.

In [ ]:
correlation_rho, p_value_spearman = stats.spearmanr(df['bio_12'], df['Beak.Length_Culmen'])

print(f"Spearman rho: {correlation_rho:.4f}")
print(f"p-value: {p_value_spearman:.4e}")

Since p < 0.05, the null hypothesis (H0) is rejected. There is a statistically significant relationship between Beak Length and Precipitation. To illustrate the relationship, a scatter plot is created (with log(Length) for better visualization).

In [ ]:
df['log_BeakLength'] = np.log10(df['Beak.Length_Culmen'])

rho, p = stats.spearmanr(df['bio_12'], df['log_BeakLength'], nan_policy='omit')

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

sns.regplot(
    x='bio_12', 
    y='log_BeakLength', 
    data=df, 
    ax=ax, 
    scatter_kws={'alpha': 0.1, 's': 10}, 
    line_kws={'color': 'red', 'lw': 2, 'label': f'ρ = {rho:.3f}, p = {p:.2e}'}
)

ax.set_title(f"Log(Beak Length) vs. Precipitation", fontsize=14)
ax.set_xlabel(f"Precipitation (mm)")
ax.set_ylabel(f"Log(Length) (log(mm))") 
ax.legend(loc='upper right', fontsize=10)

plt.show()